In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a') # max
#     ###################################################### ~Q(s,~a)= r # min
#     ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
#     targetQs = rewards + (gamma * aQs2_logits)
#     eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:16.0000 R:16.0000 rate:0.0320 aloss:0.7002 eloss:2.9667 aloss2:1.9316 exploreP:0.9984
Episode:1 meanR:19.0000 R:22.0000 rate:0.0440 aloss:0.6945 eloss:2.9355 aloss2:1.9303 exploreP:0.9962
Episode:2 meanR:21.6667 R:27.0000 rate:0.0540 aloss:0.6941 eloss:2.9423 aloss2:1.9285 exploreP:0.9936
Episode:3 meanR:21.7500 R:22.0000 rate:0.0440 aloss:0.6937 eloss:2.9429 aloss2:1.9317 exploreP:0.9914
Episode:4 meanR:22.6000 R:26.0000 rate:0.0520 aloss:0.6929 eloss:2.9322 aloss2:1.9387 exploreP:0.9889
Episode:5 meanR:21.1667 R:14.0000 rate:0.0280 aloss:0.6890 eloss:2.9338 aloss2:1.9213 exploreP:0.9875
Episode:6 meanR:20.7143 R:18.0000 rate:0.0360 aloss:0.6941 eloss:2.9046 aloss2:1.9534 exploreP:0.9857
Episode:7 meanR:19.7500 R:13.0000 rate:0.0260 aloss:0.6924 eloss:3.0231 aloss2:1.8666 exploreP:0.9845
Episode:8 meanR:21.4444 R:35.0000 rate:0.0700 aloss:0.6907 eloss:2.9511 aloss2:1.9179 exploreP:0.9811
Episode:9 meanR:21.3000 R:20.0000 rate:0.0400 aloss:0.6897 eloss:2.9134 aloss2:1.9

Episode:80 meanR:21.6420 R:23.0000 rate:0.0460 aloss:0.7317 eloss:1.9860 aloss2:3.1822 exploreP:0.8408
Episode:81 meanR:21.6463 R:22.0000 rate:0.0440 aloss:0.7236 eloss:1.9930 aloss2:3.1682 exploreP:0.8390
Episode:82 meanR:21.7590 R:31.0000 rate:0.0620 aloss:0.7122 eloss:1.9857 aloss2:3.1567 exploreP:0.8364
Episode:83 meanR:21.7262 R:19.0000 rate:0.0380 aloss:0.7065 eloss:1.9894 aloss2:3.1685 exploreP:0.8349
Episode:84 meanR:21.7647 R:25.0000 rate:0.0500 aloss:0.7080 eloss:1.9817 aloss2:3.1606 exploreP:0.8328
Episode:85 meanR:21.7209 R:18.0000 rate:0.0360 aloss:0.7203 eloss:1.9579 aloss2:3.1869 exploreP:0.8313
Episode:86 meanR:21.6322 R:14.0000 rate:0.0280 aloss:0.7657 eloss:1.9940 aloss2:3.1192 exploreP:0.8302
Episode:87 meanR:21.8295 R:39.0000 rate:0.0780 aloss:0.6946 eloss:1.9760 aloss2:3.2212 exploreP:0.8270
Episode:88 meanR:21.8764 R:26.0000 rate:0.0520 aloss:0.7080 eloss:1.9973 aloss2:3.1234 exploreP:0.8249
Episode:89 meanR:21.8111 R:16.0000 rate:0.0320 aloss:0.6909 eloss:1.9672 

Episode:160 meanR:19.7300 R:19.0000 rate:0.0380 aloss:0.7129 eloss:1.6475 aloss2:4.2974 exploreP:0.7241
Episode:161 meanR:19.5000 R:11.0000 rate:0.0220 aloss:0.7162 eloss:1.6354 aloss2:4.3188 exploreP:0.7233
Episode:162 meanR:19.4400 R:22.0000 rate:0.0440 aloss:0.7119 eloss:1.6025 aloss2:4.3596 exploreP:0.7217
Episode:163 meanR:19.3200 R:18.0000 rate:0.0360 aloss:0.7269 eloss:1.6129 aloss2:4.2934 exploreP:0.7205
Episode:164 meanR:19.5300 R:35.0000 rate:0.0700 aloss:0.7116 eloss:1.5952 aloss2:4.3147 exploreP:0.7180
Episode:165 meanR:19.4000 R:10.0000 rate:0.0200 aloss:0.7139 eloss:1.6046 aloss2:4.7308 exploreP:0.7173
Episode:166 meanR:19.2900 R:10.0000 rate:0.0200 aloss:0.6892 eloss:1.5931 aloss2:4.5366 exploreP:0.7166
Episode:167 meanR:19.4300 R:29.0000 rate:0.0580 aloss:0.7444 eloss:1.6287 aloss2:4.7710 exploreP:0.7145
Episode:168 meanR:19.3100 R:10.0000 rate:0.0200 aloss:0.6932 eloss:1.6313 aloss2:4.3181 exploreP:0.7138
Episode:169 meanR:19.2400 R:16.0000 rate:0.0320 aloss:0.6871 elo

Episode:239 meanR:16.7100 R:23.0000 rate:0.0460 aloss:0.7276 eloss:1.4897 aloss2:5.5645 exploreP:0.6393
Episode:240 meanR:16.6200 R:16.0000 rate:0.0320 aloss:0.7493 eloss:1.4851 aloss2:5.4078 exploreP:0.6383
Episode:241 meanR:16.6700 R:23.0000 rate:0.0460 aloss:0.7618 eloss:1.4535 aloss2:5.3238 exploreP:0.6368
Episode:242 meanR:16.6500 R:15.0000 rate:0.0300 aloss:0.7239 eloss:1.5081 aloss2:5.6339 exploreP:0.6359
Episode:243 meanR:16.6300 R:16.0000 rate:0.0320 aloss:0.7445 eloss:1.4890 aloss2:5.6615 exploreP:0.6349
Episode:244 meanR:16.6400 R:17.0000 rate:0.0340 aloss:0.7271 eloss:1.4952 aloss2:5.2423 exploreP:0.6338
Episode:245 meanR:16.6100 R:10.0000 rate:0.0200 aloss:0.6986 eloss:1.4423 aloss2:5.2537 exploreP:0.6332
Episode:246 meanR:16.5400 R:10.0000 rate:0.0200 aloss:0.7220 eloss:1.5069 aloss2:5.4431 exploreP:0.6326
Episode:247 meanR:16.6100 R:18.0000 rate:0.0360 aloss:0.7261 eloss:1.4716 aloss2:5.5454 exploreP:0.6315
Episode:248 meanR:16.3300 R:18.0000 rate:0.0360 aloss:0.7515 elo

Episode:318 meanR:33.5400 R:35.0000 rate:0.0700 aloss:0.7056 eloss:1.4463 aloss2:6.0223 exploreP:0.4751
Episode:319 meanR:33.9800 R:54.0000 rate:0.1080 aloss:0.7044 eloss:1.4416 aloss2:6.0698 exploreP:0.4726
Episode:320 meanR:34.4800 R:59.0000 rate:0.1180 aloss:0.7132 eloss:1.4199 aloss2:6.1708 exploreP:0.4699
Episode:321 meanR:34.5700 R:25.0000 rate:0.0500 aloss:0.6920 eloss:1.4474 aloss2:6.1602 exploreP:0.4687
Episode:322 meanR:35.2400 R:89.0000 rate:0.1780 aloss:0.7075 eloss:1.4186 aloss2:6.1398 exploreP:0.4647
Episode:323 meanR:35.9500 R:80.0000 rate:0.1600 aloss:0.7057 eloss:1.4053 aloss2:6.2405 exploreP:0.4611
Episode:324 meanR:36.1600 R:38.0000 rate:0.0760 aloss:0.7365 eloss:1.3859 aloss2:6.1882 exploreP:0.4594
Episode:325 meanR:37.3600 R:130.0000 rate:0.2600 aloss:0.7034 eloss:1.3922 aloss2:6.2216 exploreP:0.4535
Episode:326 meanR:37.4800 R:23.0000 rate:0.0460 aloss:0.7164 eloss:1.3992 aloss2:6.2895 exploreP:0.4525
Episode:327 meanR:37.7600 R:51.0000 rate:0.1020 aloss:0.6979 el

...

Episode:713 meanR:49.2000 R:113.0000 rate:0.2260 aloss:0.7576 eloss:1.3403 aloss2:6.3487 exploreP:0.1707
Episode:714 meanR:49.5400 R:45.0000 rate:0.0900 aloss:0.6895 eloss:1.3091 aloss2:6.3599 exploreP:0.1700
Episode:715 meanR:51.3300 R:190.0000 rate:0.3800 aloss:0.7161 eloss:1.3116 aloss2:6.3408 exploreP:0.1670
Episode:716 meanR:51.5300 R:36.0000 rate:0.0720 aloss:0.7082 eloss:1.3837 aloss2:6.3198 exploreP:0.1664
Episode:717 meanR:51.7400 R:34.0000 rate:0.0680 aloss:0.6939 eloss:1.3360 aloss2:6.3031 exploreP:0.1659
Episode:718 meanR:51.9300 R:28.0000 rate:0.0560 aloss:0.7364 eloss:1.3108 aloss2:6.2702 exploreP:0.1655
Episode:719 meanR:52.2500 R:44.0000 rate:0.0880 aloss:0.7368 eloss:1.3147 aloss2:6.2514 exploreP:0.1648
Episode:720 meanR:52.5100 R:36.0000 rate:0.0720 aloss:0.7404 eloss:1.3071 aloss2:6.2003 exploreP:0.1642
Episode:721 meanR:53.1400 R:76.0000 rate:0.1520 aloss:0.7156 eloss:1.3335 aloss2:6.2495 exploreP:0.1631
Episode:722 meanR:53.9900 R:94.0000 rate:0.1880 aloss:0.7358 e

Episode:792 meanR:51.6700 R:52.0000 rate:0.1040 aloss:0.7640 eloss:1.3345 aloss2:6.3818 exploreP:0.1191
Episode:793 meanR:51.4300 R:28.0000 rate:0.0560 aloss:0.7355 eloss:1.3167 aloss2:6.4935 exploreP:0.1188
Episode:794 meanR:51.1700 R:36.0000 rate:0.0720 aloss:0.7581 eloss:1.3547 aloss2:6.4530 exploreP:0.1184
Episode:795 meanR:51.4000 R:40.0000 rate:0.0800 aloss:0.7304 eloss:1.3106 aloss2:6.4667 exploreP:0.1179
Episode:796 meanR:51.9500 R:96.0000 rate:0.1920 aloss:0.7279 eloss:1.2994 aloss2:6.4495 exploreP:0.1169
Episode:797 meanR:52.1000 R:52.0000 rate:0.1040 aloss:0.7372 eloss:1.3086 aloss2:6.5097 exploreP:0.1164
Episode:798 meanR:51.8300 R:38.0000 rate:0.0760 aloss:0.7488 eloss:1.3044 aloss2:6.4757 exploreP:0.1160
Episode:799 meanR:51.7700 R:25.0000 rate:0.0500 aloss:0.7243 eloss:1.2542 aloss2:6.4874 exploreP:0.1157
Episode:800 meanR:51.6400 R:34.0000 rate:0.0680 aloss:0.7823 eloss:1.2777 aloss2:6.4548 exploreP:0.1153
Episode:801 meanR:51.4200 R:34.0000 rate:0.0680 aloss:0.7411 elo

Episode:871 meanR:47.7600 R:30.0000 rate:0.0600 aloss:0.8049 eloss:1.2951 aloss2:6.5191 exploreP:0.0836
Episode:872 meanR:48.1700 R:77.0000 rate:0.1540 aloss:0.7827 eloss:1.3011 aloss2:6.5550 exploreP:0.0830
Episode:873 meanR:48.0400 R:34.0000 rate:0.0680 aloss:0.7744 eloss:1.3413 aloss2:6.5358 exploreP:0.0828
Episode:874 meanR:48.2000 R:41.0000 rate:0.0820 aloss:0.7820 eloss:1.3381 aloss2:6.5286 exploreP:0.0825
Episode:875 meanR:48.1400 R:31.0000 rate:0.0620 aloss:0.7501 eloss:1.3172 aloss2:6.5426 exploreP:0.0823
Episode:876 meanR:47.9500 R:33.0000 rate:0.0660 aloss:0.7668 eloss:1.2668 aloss2:6.5529 exploreP:0.0820
Episode:877 meanR:48.1500 R:54.0000 rate:0.1080 aloss:0.7787 eloss:1.3139 aloss2:6.5289 exploreP:0.0816
Episode:878 meanR:48.3100 R:66.0000 rate:0.1320 aloss:0.7692 eloss:1.3298 aloss2:6.5203 exploreP:0.0812
Episode:879 meanR:48.4200 R:56.0000 rate:0.1120 aloss:0.7799 eloss:1.2657 aloss2:6.5255 exploreP:0.0808
Episode:880 meanR:48.4600 R:44.0000 rate:0.0880 aloss:0.7903 elo

Episode:950 meanR:47.9900 R:59.0000 rate:0.1180 aloss:0.8052 eloss:1.3194 aloss2:6.6231 exploreP:0.0615
Episode:951 meanR:47.7000 R:39.0000 rate:0.0780 aloss:0.8030 eloss:1.3023 aloss2:6.6006 exploreP:0.0613
Episode:952 meanR:47.4500 R:36.0000 rate:0.0720 aloss:0.8089 eloss:1.2833 aloss2:6.5976 exploreP:0.0611
Episode:953 meanR:47.3200 R:64.0000 rate:0.1280 aloss:0.7771 eloss:1.2754 aloss2:6.6436 exploreP:0.0607
Episode:954 meanR:47.6400 R:57.0000 rate:0.1140 aloss:0.8237 eloss:1.3107 aloss2:6.6052 exploreP:0.0605
Episode:955 meanR:47.0900 R:31.0000 rate:0.0620 aloss:0.8202 eloss:1.3579 aloss2:6.5841 exploreP:0.0603
Episode:956 meanR:47.2600 R:46.0000 rate:0.0920 aloss:0.7891 eloss:1.3290 aloss2:6.5778 exploreP:0.0601
Episode:957 meanR:47.4900 R:49.0000 rate:0.0980 aloss:0.8182 eloss:1.3345 aloss2:6.5585 exploreP:0.0598
Episode:958 meanR:46.9900 R:48.0000 rate:0.0960 aloss:0.8306 eloss:1.2215 aloss2:6.5958 exploreP:0.0596
Episode:959 meanR:46.9100 R:73.0000 rate:0.1460 aloss:0.8225 elo

Episode:1029 meanR:51.0200 R:37.0000 rate:0.0740 aloss:0.6822 eloss:1.3039 aloss2:6.5099 exploreP:0.0443
Episode:1030 meanR:51.1900 R:55.0000 rate:0.1100 aloss:0.6872 eloss:1.2924 aloss2:6.5401 exploreP:0.0441
Episode:1031 meanR:51.8000 R:110.0000 rate:0.2200 aloss:0.7128 eloss:1.3171 aloss2:6.5221 exploreP:0.0437
Episode:1032 meanR:51.9900 R:44.0000 rate:0.0880 aloss:0.6701 eloss:1.3725 aloss2:6.5091 exploreP:0.0436
Episode:1033 meanR:51.9700 R:46.0000 rate:0.0920 aloss:0.7230 eloss:1.3134 aloss2:6.5258 exploreP:0.0434
Episode:1034 meanR:51.8400 R:47.0000 rate:0.0940 aloss:0.7126 eloss:1.2872 aloss2:6.5330 exploreP:0.0432
Episode:1035 meanR:51.6600 R:27.0000 rate:0.0540 aloss:0.6762 eloss:1.3499 aloss2:6.5498 exploreP:0.0432
Episode:1036 meanR:51.7100 R:33.0000 rate:0.0660 aloss:0.7063 eloss:1.2531 aloss2:6.5616 exploreP:0.0430
Episode:1037 meanR:52.0800 R:70.0000 rate:0.1400 aloss:0.6936 eloss:1.3273 aloss2:6.5649 exploreP:0.0428
Episode:1038 meanR:51.5600 R:55.0000 rate:0.1100 aloss

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
